In [1]:
import numpy as np
import pandas as pd
import random as rnd

In [2]:
class SVM():
    def __init__(self,X,y,C = 1,b = 0 ,kernel_type="linear",epsilon = 0.001,max_itera = 10000):
        self.kernels = {'linear': self.kernel_linear,'quadratic': self.kernel_quadratic}
        self.rows,self.columns = X.shape[0],X.shape[1]
        self.alpha = np.zeros( self.rows )
        self.w = np.zeros(self.columns)
        self.kernel_type = kernel_type
        self.max_itera = max_itera
        self.epsilon = epsilon
        self.C = C
        self.b = b
        self.X = X
        self.y = y
        
    def kernel_linear(self,x1,x2):
        return np.dot(x1,x2)
    
    def g(self,w,b,i):
        return np.sign(np.dot(w,self.X[i].T) + b)
    
    def E_i(self,w,b,i):
        return self.g(w,b,i) - self.y[ i ]
    
    def W(self,alpha):
        return np.dot(np.multiply(alpha,self.y),self.X)
    
    def get_index_j(self,a,b,z):
        i = z
        cnt = 0
        while i == z and cnt < 1000:
            i = rnd.randint(a,b)
            cnt = cnt + 1
        return i
    
    def get_L_H(self, alpha_j, alpha_i, y_j, y_i):
        if (y_i != y_j):
            return (max(0, alpha_j - alpha_i), min(self.C, self.C - alpha_i + alpha_j))
        else:
            return (max(0, alpha_i + alpha_j - self.C), min(self.C, alpha_i + alpha_j))
        
    def kernel_quadratic():
        pass
    
    def fit(self):
        passes = 0
        kernel = self.kernels[self.kernel_type]
        while passes < self.max_itera:
            num_changed_alpha = 0
            for i in range( self.rows ):
                E_i = self.E_i( self.W(self.alpha),self.b,i )
                if(y[ i ]*E_i < -self.epsilon and self.alpha[ i ] < self.C) or ( y[ i ]*E_i > self.epsilon and self.alpha[ i ] > 0):
                    j = self.get_index_j( 0,self.rows-1,i )
                    E_j = self.E_i( self.W(self.alpha),self.b,j )
                    
                    alpha_old = np.copy(self.alpha) 
                    L,H = self.get_L_H( self.alpha[j],self.alpha[i],self.y[j],self.y[i] )
                    if L == H:
                        continue
                    eta = -kernel(self.X[ i ],self.X[ i ]) -kernel(self.X[ j ],self.X[ j ]) + 2 * kernel(self.X[ i ],self.X[ j ])
                    if eta >= 0:
                        continue
                    
                    self.alpha[ j ] = self.alpha[ j ] - self.y[j]*((E_i - E_j)/eta)
                    self.alpha[ j ] = max(self.alpha[ j ],L)
                    self.alpha[ j ] = min(self.alpha[ j ],H)
                    
                    if abs( self.alpha[ j ] - alpha_old[ j ]) < 1e-5:
                        continue
                        
                    self.alpha[ i ] = alpha_old[ i ] + self.y[ i ]*self.y[ j ] * (alpha_old[ j ] - self.alpha[ j ])
                    
                    b1 = self.b - E_i - self.y[ i ]*(self.alpha[i] - alpha_old[i])*kernel(self.X[ i ], self.X[ i ]) - self.y[ j ]*(self.alpha[ j ] - alpha_old[ j ])*kernel(self.X[ i ], self.X[ j ])
                    b2 = self.b - E_j - self.y[ i ]*(self.alpha[i] - alpha_old[i])*kernel(self.X[ i ], self.X[ j ]) - self.y[ j ]*(self.alpha[ j ] - alpha_old[ j ])*kernel(self.X[ j ], self.X[ j ])
                    
                    if((0 < self.alpha[i] and self.alpha[i] < self.C) and (0 < self.alpha[j] and self.alpha[j] < self.C)):
                        self.b = (b1 + b2)/2
                    elif((0 < self.alpha[i] and self.alpha[i] < self.C)):
                        self.b = b1
                    elif ((0 < self.alpha[j] and self.alpha[j] < self.C)):
                        self.b = b2
                    else:
                        self.b = (b1 + b2)/2
                    
                    num_changed_alpha += 1
                if num_changed_alpha == 0:
                    passes += 1
                else:
                    passes = 0
                    
            self.w = self.W(self.alpha)
            
            alpha_idx = np.where(self.alpha > 0)[0]
            support_vectors = self.X[alpha_idx]
            return support_vectors
        
        
    def predict(self,X):
        return np.sign(np.dot(self.w, X.T) + self.b).astype(int)
    
    def precision(self,X,y):
        predict = self.predict(X)
        predict = np.where(predict==1,1,0)
        TP = np.sum(np.where(y==predict,1,0))
        return TP/sum(predict)
    
    def recall(self,X,y):
        TP_FN = sum(np.where(y==1,1,0))
        predict = self.predict(X)
        predict = np.where(predict==1,1,0)
        TP = np.sum(np.where(y==predict,1,0))
        return TP/TP_FN            

In [3]:
data=pd.read_csv("~/Binary-classification-dataset-master/data0/data.csv", names = ['label','x1', 'x2'])

In [4]:
X=data.values[:,1:]
y=data.values[:,0]

In [5]:
model=SVM(X,y)

In [6]:
model.fit()

array([[2.4711, 5.8097],
       [1.169 , 0.9008],
       [4.0693, 6.2652],
       [4.7598, 7.1508],
       [5.7068, 8.3148],
       [4.5822, 2.7227],
       [2.9841, 1.6852],
       [6.7722, 6.6953],
       [7.1471, 6.6194],
       [7.6206, 8.5931],
       [5.0163, 2.3684]])

In [7]:
model.precision(X,y)

1.0

In [8]:
model.recall(X,y)

0.94